In [1]:
import pandas as pd 
import numpy as np 
import os
from tqdm import tqdm
from tokenizer import NLTKTokenizer
from tokenizer import RegTokenizer

In [2]:
CWD = os.getcwd()
TRAIN_DATA_PATH = os.path.join(CWD, 'data', 'trainset.csv')
VALID_DATA_PATH = os.path.join(CWD, 'data', 'validset.csv')
TEST_DATA_PATH = os.path.join(CWD, 'data', 'testset.csv')
# DICT_PATH = os.path.join(CWD, 'data', 'dictionary.pkl')
DICT_PATH = os.path.join(CWD, 'data', 'dictionary_reg.pkl')
PAD_TOKEN = '[PAD]'
PAD_TOKEN_ID = 0
EOS_TOKEN = '[EOS]'
EOS_TOKEN_ID = 3

# Tokenizer = NLTKTokenizer(  pad_token=PAD_TOKEN,
#                             pad_token_id=PAD_TOKEN_ID,
#                             eos_token=EOS_TOKEN,
#                             eos_token_id=EOS_TOKEN_ID  )


Tokenizer_Reg = RegTokenizer(   pad_token=PAD_TOKEN,
                                pad_token_id=PAD_TOKEN_ID,
                                eos_token=EOS_TOKEN,
                                eos_token_id=EOS_TOKEN_ID  )

In [3]:
def GenDict(train, valid):
    # global Tokenizer
    global Tokenizer_Reg
    if os.path.exists(DICT_PATH):
        # Tokenizer = NLTKTokenizer.load_from_file(DICT_PATH)
        Tokenizer_Reg = RegTokenizer.load_from_file(DICT_PATH)
    else:
        for item in tqdm(train['Abstract'], desc='Train set'):
            # Tokenizer.build_dict([item])
            Tokenizer_Reg.build_dict([item], min_count=5)
        for item in tqdm(valid['Abstract'], desc='Valid set'):
            # Tokenizer.build_dict([item])
            Tokenizer_Reg.build_dict([item], min_count=5)
        # Tokenizer.save_to_file(DICT_PATH)
        Tokenizer_Reg.save_to_file(DICT_PATH)

In [33]:
train = pd.read_csv(TRAIN_DATA_PATH, dtype=str)
valid = pd.read_csv(VALID_DATA_PATH, dtype=str)
test = pd.read_csv(TEST_DATA_PATH, dtype=str)
print('Generate relative dictionary')
GenDict(train, valid)

Generate relative dictionary


In [8]:
def get_OOV(word_dict, wordvector_path, embedding_dim):
    embeddings_index = {}
    f = open(wordvector_path)
    for line in f:
        values = line.replace(',','').split()
        token = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[token] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))

    # max_words = Tokenizer.vocab_size()
    # embedding_matrix = np.random.randn(max_words, embedding_dim)

    OOV_list = []
    for token, index in word_dict.items():
        embedding_vector = embeddings_index.get(token)
        # if embedding_vector is not None:
            # embedding_matrix[index] = embedding_vector
        if embedding_vector is None:
            OOV_list.append(token)

    return OOV_list

In [22]:
# test.dropna(inplace=True)

In [37]:
len(test)

131166

In [43]:
test.isna()

,Abstract
0,False
1,False
2,False
3,False
4,False
...,...
131161,False
131162,False
131163,False
131164,False


In [42]:
test.isna().iloc[26369,:]

Abstract    True
Name: 26369, dtype: bool

In [44]:
test['Abstract'][test.isna()['Abstract']]

26369     NaN
58459     NaN
61649     NaN
122529    NaN
126428    NaN
Name: Abstract, dtype: object

In [6]:
# oov_list = get_OOV(Tokenizer.get_token_to_id(), f'glove/glove_512d.txt', 512)
oov_list = get_OOV(Tokenizer_Reg.get_token_to_id(), f'glove/glove_512d.txt', 512)

Found 1425892 word vectors.


In [7]:
len(oov_list)

6

In [8]:
with open('oov_words_RegTokenize.txt', 'w') as f:
    for word in oov_list:
        f.write(word)
        f.write('\n')